Write a function to convert a date to adatetime data type.

Write a function to change a datetime to UTC.

Write a function to parse a date column into 6 additional columns (while keeping the original date): year, quarter, month, day of month, day of week, weekend vs. weekday

Create a new dataframe that aggregates the sales (sum and median) by day of week.

Explore the pandas DataFrame.diff() function. Create a new column that is the result of the current sales - the previous days sales.

Write a function to set the index to be the datetime variable.

In [183]:
# environment set-up:

import pandas as pd
import requests
import acquire
import summarize

#acquire: 
df = acquire.get_all_stuff()

In [184]:
df.info()
#Walkthrough method: 

# datetime_format = '%a, %d %b %Y %H:%M:%S %Z'
# pd.to_datetime(df.sale_date.head(), format=datetime_format, utc=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913000 entries, 0 to 912999
Data columns (total 14 columns):
item_id          913000 non-null int64
sale_amount      913000 non-null float64
sale_date        913000 non-null object
sale_id          913000 non-null int64
store_id         913000 non-null int64
store_address    913000 non-null object
store_city       913000 non-null object
store_state      913000 non-null object
store_zipcode    913000 non-null int64
item_brand       913000 non-null object
item_name        913000 non-null object
item_price       913000 non-null float64
item_upc12       913000 non-null int64
item_upc14       913000 non-null int64
dtypes: float64(2), int64(6), object(6)
memory usage: 104.5+ MB


In [173]:
# dfwlkthrgh.set_index('sale_date', inplace=True)
# dfwlkthrgh.tz_localize('utc')

  Write a function to convert a date to a datetime data type.

Write a function to parse a date column into 6 additional columns (while keeping the original date): year, quarter, month, day of month, day of week, weekend vs. weekday

In [174]:
def to_datetime_utc(df):
    s = df['sale_date'].apply(lambda x: x.split(','))
    df['weekday'] = s.apply(lambda x: x[0])
    df['sale_date'] = s.apply(lambda x: x[1])
    df['sale_date'] = pd.to_datetime(df['sale_date'], infer_datetime_format=True, utc=True)
    return df

In [175]:
def make_cols(df):
    df['year'] = df.sale_date.dt.year
    df['quarter'] = df.sale_date.dt.quarter
    df['month'] = df.sale_date.dt.month
    df['day_of_month'] = df.sale_date.dt.day
    df['is_weekday'] = df.sale_date.dt.weekday.apply(lambda x: 1 if x >= 5 else 0)
    return df

In [178]:
df.is_weekday.value_counts(dropna=False)

0    652000
1    261000
Name: is_weekday, dtype: int64

### Create a new dataframe that aggregates the sales (sum and median) by day of week.

In [ ]:
def make_new_sales_amounts(df):
    df['sales_total'] = df.sale_amount * df.item_price
    df.rename(columns={'sale_amount': 'quantity'}, inplace=True)
    return df
df = make_new_sales_amounts(df)
df.head()

In [182]:
sum_by_weekday = df[['weekday', 'sales_total']].groupby(df.weekday).sum()
# mean_by_weekday = df[['weekday', 'sales_total']].groupby(df.weekday).mean()
print(sum_by_weekday)
# print(mean_by_weekday)

          sales_total
weekday              
Fri      3.868773e+07
Mon      2.896118e+07
Sat      4.116177e+07
Sun      4.359486e+07
Thu      3.628727e+07
Tue      3.383954e+07
Wed      3.392832e+07


### Explore the pandas DataFrame.diff() function. Create a new column that is the result of the current sales - the previous days sales.

In [120]:
df.sale_date.nunique()

1826

In [129]:
df_sum_date = df.groupby(df.sale_date, as_index=False).sum()

In [130]:
df_sum_date['prev_day_diff'] = df_sum_date['sales_total'].diff()

In [133]:
df_sum_date[['sale_date', 'sales_total', 'prev_day_diff']].head(10)

,sale_date,sale_amount,prev_day_diff
0,2013-01-01 00:00:00+00:00,13696.0,NaN
1,2013-01-02 00:00:00+00:00,13678.0,-18.0
2,2013-01-03 00:00:00+00:00,14488.0,810.0
3,2013-01-04 00:00:00+00:00,15677.0,1189.0
4,2013-01-05 00:00:00+00:00,16237.0,560.0
5,2013-01-06 00:00:00+00:00,17291.0,1054.0
6,2013-01-07 00:00:00+00:00,11769.0,-5522.0
7,2013-01-08 00:00:00+00:00,13560.0,1791.0
8,2013-01-09 00:00:00+00:00,13878.0,318.0
9,2013-01-10 00:00:00+00:00,14642.0,764.0


### Write a function to set the index to be the datetime variable.

In [135]:
def re_index_by_time(df):
    df.set_index('sale_date', inplace=True)